# Setup

In [24]:
!pip install 'crewai[tools]' -q
!pip install youtube-transcript-api -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 622.3/622.3 kB 7.9 MB/s eta 0:00:00


In [2]:
# Retrieve the API keys securely from Google Colab's user data
from google.colab import userdata
openai_api_key = userdata.get('ai_agents_openai')
serper_api_key = userdata.get('serper_api')

In [3]:
import os
# Set the API keys as environment variables for accessibility
os.environ['OPENAI_API_KEY'] = openai_api_key
os.environ['SERPER_API_KEY'] = serper_api_key

In [26]:
from crewai import Agent, Crew, Process, Task
from langchain_openai import ChatOpenAI
from IPython.display import display, Markdown
from crewai_tools import ScrapeWebsiteTool, SerperDevTool, YoutubeChannelSearchTool, YoutubeVideoSearchTool
from pydantic import BaseModel

In [5]:
# Define the LLM
llm = ChatOpenAI(model_name='gpt-4o', temperature=0.5)

# Data Inputs

In [6]:
brand_info = {
    "name": "Zero to Mastery",
    "description": "An online tech academy that offers comprehensive courses and a community for learners seeking in-demand software and tech skills.",
    "brand_values": "Quality education, community-driven growth, career-focused learning, and up-to-date tech stacks.",
    "ideal_partner": "A YouTube channel focused on programming, software engineering, or tech career advice with an audience of motivated learners."
}

In [7]:
# Define a single YouTube lead for demonstration
youtube_lead = {
    "name": "Theo - t3․gg",
    "channel_url": "https://www.youtube.com/@t3dotgg"
}

# Research

In [16]:
# Build a Research Agent
research_agent = Agent(
    role = "Influencer researcher",
    goal = f"Research ybout {youtube_lead['name']}'s channel {youtube_lead['channel_url']} to assess their suitability for {brand_info['name']}",
    backstory = f"You are an expert in Influencer partnerships that align with {brand_info['name']} and its respective values: {brand_info['brand_values']}",
    llm = llm,
    tools = [SerperDevTool(), ScrapeWebsiteTool(), YoutubeChannelSearchTool(), YoutubeVideoSearchTool()]
)

In [20]:
# Define the google research task
channel_research_task = Task(
    description = f"""
    Research the channel {youtube_lead['name']} at {youtube_lead['channel_url']}:
    1. Identify channel focus and content type.
    2. Determine approximate subscriber count and engagement.
    3. Check for audience alignment with {brand_info['name']}""",
    expected_output = "A research report on the channel's key attributes and audience",
    agent = research_agent
)

In [12]:
# Define the research videos task
videos_research_task = Task(
    description = f"""
    Research the most popular videos from channel {youtube_lead['name']} at {youtube_lead['channel_url']}.
    Summarise their topics and analyse the audiences.""",
    expected_output="A research report on the most popular videos and their topics",
    agent = research_agent)

In [13]:
# Define a sentiment on the channel via Google searches
sentiment_task = Task(
    description = f"""
    Conduct a sentiment analysis via Google Search on the channel {youtube_lead['name']} at {youtube_lead['channel_url']}.
    Look for user feedbacks, resviews, or discussions that indicate general sentiment.
    Search for instance on reddit.""",
    expected_output= "A sentiment analysis report on the channel's",
    agent = research_agent)

# AI Scoring Agent

In [25]:
# Developping the Scoring Agent
scoring_agent =  Agent(
    role = "Partnership Scoring Agent",
    goal = f"""Score the partnership between {brand_info['name']} and {youtube_lead['name']}.
    Assess based on the {brand_info['brand_values']}""",
    backstory = f"You are an expert in Influencer Marking working for {brand_info['name']}",
    llm = llm,
)

In [33]:
# Create a Base Model Class to provide a score
class Score(BaseModel):
    score: int
    justification: str

In [34]:
# Define the Scoring Task
scoring_task = Task(
    description = f"""
    Score the partnership between {brand_info['name']} and {youtube_lead['name']}""",
    expected_output = "A score between 1 and 10 and justification for the scoring. An explanation of the score indicating what fits the brand and what does not",
    agent = scoring_agent,
    context = [channel_research_task, videos_research_task, sentiment_task],
    output_pydantic= Score
)


# Build the AI Crew

In [35]:
# Assemble the AI crew
crew = Crew(
    agents = [research_agent, scoring_agent],
    tasks = [channel_research_task, videos_research_task, sentiment_task, scoring_task],
    process = Process.sequential,
    verbose = True
)
result = crew.kickoff()

# Agent: Influencer researcher
## Task: 
    Research the channel Theo - t3․gg at https://www.youtube.com/@t3dotgg:
    1. Identify channel focus and content type.
    2. Determine approximate subscriber count and engagement.
    3. Check for audience alignment with Zero to Mastery


# Agent: Influencer researcher
## Using tool: Search the internet with Serper
## Tool Input: 
"{\"search_query\": \"Theo - t3\\u2024gg YouTube channel overview\"}"
## Tool Output: 
{'searchParameters': {'q': 'Theo - t3․gg YouTube channel overview', 'type': 'search', 'num': 10, 'engine': 'google'}, 'organic': [{'title': 'Theo - t3․gg - YouTube', 'link': 'https://www.youtube.com/@t3dotgg', 'snippet': "Hi, I'm a software dev nerd mostly known for full stack TypeScript stuff. Check out me and my stack at t3.gg", 'position': 1}, {'title': "theo - builder who's also on youtube", 'link': 'https://t3.gg/', 'snippet': 'theo (aka t3, t3dotgg) is a youtuber/ceo/dev. built t3 chat, ping.gg, t3 stack and more.', 'posit

Inserting batches in chromadb: 100%|██████████| 1/1 [00:00<00:00,  2.06it/s]




# Agent: Influencer researcher
## Using tool: Search a Youtube Channels content
## Tool Input: 
"{\"search_query\": \"overview\", \"youtube_channel_handle\": \"@t3dotgg\"}"
## Tool Output: 
Relevant Content:
@t3dotgg


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: Search the internet with Serper
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the internet', 'type': 'str'}}
Tool Description: A tool that can be used to search the internet with a search_query. Supports different search types: 'search' (default), 'news'
Tool Name: Read website content
Tool Arguments: {'website_url': {'description': 'Mandatory website url to read the file', 'type': 'str'}}
Tool Description: A tool that can be used to read a website content.
Tool Name: Search a Youtube Channels content
Tool Arguments: {'search_query': {'description': 'Mandatory search query you want to use to search the Yo

In [44]:

import json
# Accessing the final outputs
channel_research_output = result.tasks_output[0].raw
videos_research_output = result.tasks_output[1].raw
sentiment_output = result.tasks_output[2].raw
scoring_output = result.tasks_output[3].raw

print("Channel Research Output:")
display(Markdown(channel_research_output))

print("\nVideos Research Output:")
display(Markdown(videos_research_output))

print("\nSentiment Analysis Output:")
display(Markdown(sentiment_output))

print("\nScoring Output:")
display(Markdown(scoring_output))

Channel Research Output:


Research Report on Theo - t3․gg YouTube Channel

1. **Channel Focus and Content Type:**
   - Theo - t3․gg is primarily focused on technology and software development, with a particular emphasis on full stack TypeScript. The channel is known for content that explores various tech stacks, software development practices, and industry insights.

2. **Subscriber Count and Engagement:**
   - As of the latest data, the channel has approximately 392K subscribers.
   - The channel has accumulated over 59.25 million video views.
   - The engagement rate is reported at 4.26%, which is considered excellent.
   - The channel uploads an average of 6.75 videos per week, with an average video length of 37.23 minutes.

3. **Audience Alignment with Zero to Mastery:**
   - The channel's focus on full stack TypeScript and modern software development practices aligns well with Zero to Mastery's values of providing quality education and up-to-date tech stacks.
   - The channel's audience, likely composed of tech enthusiasts and software developers, aligns with Zero to Mastery's target audience who are interested in career-focused learning and community-driven growth in the tech industry.

Overall, Theo - t3․gg appears to be a suitable candidate for partnership with Zero to Mastery, given its alignment in content focus, audience, and engagement metrics.
```


Videos Research Output:


The research report on the most popular videos and their topics for Theo - t3․gg's YouTube channel is complete. The channel's focus on tech stacks, practical tutorials, and industry insights aligns well with Zero to Mastery's values and target audience.


Sentiment Analysis Output:


Based on the search results, there are mixed sentiments about Theo - t3․gg's YouTube channel. Some Reddit users appreciate Theo's genuine and engaging content, particularly for mid-to-senior engineers, highlighting his strength as an entertainer who can occasionally teach. However, there are criticisms regarding his communication style, perceived arrogance, and a tendency to present opinions as facts. Some users find his content not suitable for beginners and mention that he focuses on topics that have contributed to his success. Overall, the sentiment is mixed, with both positive and negative feedback from the community.
```


Scoring Output:


{
  "score": 8,
  "justification": "The partnership between Zero to Mastery and Theo - t3.gg scores an 8 due to the strong alignment in content focus and audience. Theo's emphasis on full stack TypeScript and modern software development practices complements Zero to Mastery's mission of providing quality education and up-to-date tech stacks. The engagement metrics are excellent, indicating a highly active and interested audience, which aligns well with Zero to Mastery's goals of community-driven growth and career-focused learning. However, the mixed sentiments about Theo's communication style and the suitability of his content for beginners slightly detract from the overall score, suggesting there may be some challenges in appealing to all segments of Zero to Mastery's audience."
}